# Combining SCE, Census, & Homes

This notebook will combine SCE utility, census tracts, and building footprints, in an attempt to create an interactive plot.

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import box
import matplotlib.pyplot as plt
import glob
import os
import folium

ModuleNotFoundError: No module named 'folium'

## 1. SCE

In [ ]:
# Load SCE
sce_circuits = gpd.read_file("../../../../capstone/electrigrid/data/utilities/sce_circuits/ICA_-_Circuit_Segments.geojson")

In [ ]:
# Set bbox to coordinates: -118.240125,33.832560,-118.119275,33.907389
la_bbox = box(minx=-118.240125, miny=33.832560, maxx=-118.119275, maxy=33.907389)
la_bbox = gpd.GeoDataFrame(geometry=[la_bbox], crs=sce_circuits.crs)
# Clip circuits to bbox
la_circuits = gpd.clip(sce_circuits, la_bbox)

In [ ]:
# fig, ax = plt.subplots()

# la_circuits.plot(ax=ax)
# plt.show()

## 2. Census

In [ ]:
ca_census = gpd.read_file("../../../../capstone/electrigrid/data/census/tl_2025_06_tract/tl_2025_06_tract.shp")
ca_census = ca_census.to_crs(sce_circuits.crs)
assert sce_circuits.crs == ca_census.crs

In [ ]:
# Clip CA census tracts
la_census = gpd.clip(ca_census, la_bbox)

In [ ]:
# fig, ax = plt.subplots()

# la_census.boundary.plot(ax=ax, edgecolor='gray', linewidth=0.8, alpha=0.3)
# la_circuits.plot(ax=ax, linewidth=0.3)
# plt.show()

## 3. Buildings

In [ ]:
#url = 'https://data.source.coop/tge-labs/globalbuildingatlas-lod1/w120_n35_w115_n30.parquet'
path = "../../../../capstone/electrigrid/data/buildings/w120_n35_w115_n30_zl_1_22.parquet"
buildings = gpd.read_parquet(path)

In [ ]:
buildings = buildings.to_crs(sce_circuits.crs)
la_buildings = gpd.clip(buildings, la_bbox)

In [ ]:
# fig, ax = plt.subplots()

# la_buildings.plot(ax=ax, column='height')

# plt.show()

**If the map isn't loading correctly, create a markdown cell below the map cell. If there already is a markdown cell, change it to a code cell, then back to a markdown cell and this should fix the map.**

In [ ]:
# Initilize folium map with specified center location
m = folium.Map(location=(33.869988, -118.179760))

circuit_popup = folium.GeoJsonPopup(fields=['objectid', 'circuit_name', 'substation_name', 'circuit_voltage'])

# Census layer
folium.GeoJson(data=la_census,
               style_function=lambda feature: {"fillColor":"gray", "color":"black"}).add_to(m)

# Buildings layer
folium.GeoJson(data=la_buildings,
               style_function=lambda feature: {"color":"red"}).add_to(m)

# Circuits layer
folium.GeoJson(data=la_circuits, popup=circuit_popup, highlight_function=lambda feature: {"color":"orange"}).add_to(m)

m